In [13]:
import pandas as pd
import re
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from tqdm import tqdm
import os
from pprint import pprint
import numpy as np

In [2]:
twitter_content = pq.read_table(
    "/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/TwitterData/df_all_tweets_en.parquet").to_pandas()

In [3]:
twitter_content.TweetContent = twitter_content.TweetContent.astype(str)

In [4]:
twitter_content = twitter_content[twitter_content.TweetContent != 'nan']

In [5]:
tqdm.pandas()
twitter_content.TweetContent = twitter_content.TweetContent.progress_apply(lambda x: re.sub(r'(@\w+\s?)|((https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?)|([\n\t]+)|(\s+)', ' ', x).strip())

100%|██████████| 69499260/69499260 [24:57<00:00, 46404.16it/s] 


In [6]:
twitter_content

,URL,Date,User,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationID,RetweetedTweet,quotedTweet,inReplyToTweetID,inReplyToUser,mentionedUsers,hashtags,TweetContent,Language
0,https://twitter.com/andya_d/status/15879267685...,2022-11-02 21:56:58+00:00,https://twitter.com/andya_d,1,0,1,0,1587725229396316160,None,None,1.5879161400340152e+18,https://twitter.com/yuroon,"[User(username='yuroon', id=29169337, displayn...",None,"Very, very nice.",en
1,https://twitter.com/andya_d/status/15875736234...,2022-11-01 22:33:42+00:00,https://twitter.com/andya_d,1,0,1,0,1587363839271739392,None,None,1.5875718109079675e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,No-show in sheer defiance? Rambo couldnt give ...,en
2,https://twitter.com/andya_d/status/15875189967...,2022-11-01 18:56:38+00:00,https://twitter.com/andya_d,0,0,2,0,1587363839271739392,None,None,1.5875182031142748e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,Grammar goofs. Toen (2007) al cheeky.,en
3,https://twitter.com/andya_d/status/15874439853...,2022-11-01 13:58:34+00:00,https://twitter.com/andya_d,0,0,0,0,1587096853858115584,None,None,1.5871065260992758e+18,https://twitter.com/ElliottArcand64,"[User(username='ElliottArcand64', id=696924884...",None,Elon must antivax it up against the WHO. That'...,en
4,https://twitter.com/andya_d/status/15872511012...,2022-11-01 01:12:06+00:00,https://twitter.com/andya_d,1,0,1,0,1586988562045599744,None,None,1.5872385848426086e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,Herum-erwappt.,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69499255,https://twitter.com/fetz_mv/status/10394790615...,2018-09-11 11:41:35+00:00,https://twitter.com/fetz_mv,0,0,0,0,1038027233519001600,None,None,1.0380272335190016e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,Do you hear yourself?,en
69499256,https://twitter.com/fetz_mv/status/10391362625...,2018-09-10 12:59:25+00:00,https://twitter.com/fetz_mv,0,0,0,0,1039111866956566528,None,None,1.0391118669565665e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,You are the joke and an embarrassment to the o...,en
69499257,https://twitter.com/fetz_mv/status/10374968680...,2018-09-06 00:25:03+00:00,https://twitter.com/fetz_mv,0,0,0,0,1037485664433070080,None,None,1.0374856644330701e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,Sorry this is not a dictatorship. Free speech!!,en
69499258,https://twitter.com/fetz_mv/status/10374732683...,2018-09-05 22:51:17+00:00,https://twitter.com/fetz_mv,0,0,0,0,1037464177269514240,None,None,1.0374641772695142e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,No! trying to save our country from you!,en


# Discard non-English Tweets

In [26]:
# Discard non-English Tweets
from ftlangdetect import detect
def detect_lang(x):
    return detect(''.join(x))['lang']

In [10]:
twitter_content = twitter_content.assign(Language='')

In [29]:
twitter_content = twitter_content.drop('Language', axis=1)

In [34]:
from tqdm.auto import tqdm
twitter_content = twitter_content.assign(Language=twitter_content.TweetContent.progress_apply(detect_lang))

100%|██████████| 76718924/76718924 [21:47<00:00, 58692.29it/s] 


  0%|          | 0/76718924 [00:00<?, ?it/s]

In [35]:
twitter_content

,URL,Date,User,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationID,RetweetedTweet,quotedTweet,inReplyToTweetID,inReplyToUser,mentionedUsers,hashtags,TweetContent,Language
0,https://twitter.com/andya_d/status/15881361587...,2022-11-03 11:49:01+00:00,https://twitter.com/andya_d,0,0,1,0,1588116012066652167,None,None,1.5881165568899482e+18,https://twitter.com/RenMar1969,"[User(username='RenMar1969', id=81675569973678...",None,Ff livestreamen naar Youtube please.,nl
1,https://twitter.com/andya_d/status/15881342362...,2022-11-03 11:41:22+00:00,https://twitter.com/andya_d,0,0,1,0,1588079268046770183,None,https://twitter.com/PIughuig/status/1588116012...,1.5880792680467702e+18,https://twitter.com/BataafseNazorg,"[User(username='BataafseNazorg', id=1431521780...",None,Het kaartenhuis stort met donderend geraas in ...,nl
2,https://twitter.com/andya_d/status/15879514022...,2022-11-02 23:34:51+00:00,https://twitter.com/andya_d,0,1,6,0,1587725229396316160,None,None,1.587932471102165e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,Alle wapguru's en achteromdenkfilosofen zitten...,nl
3,https://twitter.com/andya_d/status/15879267685...,2022-11-02 21:56:58+00:00,https://twitter.com/andya_d,1,0,1,0,1587725229396316160,None,None,1.5879161400340152e+18,https://twitter.com/yuroon,"[User(username='yuroon', id=29169337, displayn...",None,"Very, very nice.",en
4,https://twitter.com/andya_d/status/15879026348...,2022-11-02 20:21:04+00:00,https://twitter.com/andya_d,0,0,1,0,1587816298939031553,None,None,1.5878964734417674e+18,https://twitter.com/Hazehart1,"[User(username='Hazehart1', id=154838232676943...",None,Alle gekochte internet-antivax-aandachtszoeker...,nl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76718919,https://twitter.com/fetz_mv/status/10394790615...,2018-09-11 11:41:35+00:00,https://twitter.com/fetz_mv,0,0,0,0,1038027233519001600,None,None,1.0380272335190016e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,Do you hear yourself?,en
76718920,https://twitter.com/fetz_mv/status/10391362625...,2018-09-10 12:59:25+00:00,https://twitter.com/fetz_mv,0,0,0,0,1039111866956566528,None,None,1.0391118669565665e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,You are the joke and an embarrassment to the o...,en
76718921,https://twitter.com/fetz_mv/status/10374968680...,2018-09-06 00:25:03+00:00,https://twitter.com/fetz_mv,0,0,0,0,1037485664433070080,None,None,1.0374856644330701e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,Sorry this is not a dictatorship. Free speech!!,en
76718922,https://twitter.com/fetz_mv/status/10374732683...,2018-09-05 22:51:17+00:00,https://twitter.com/fetz_mv,0,0,0,0,1037464177269514240,None,None,1.0374641772695142e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,No! trying to save our country from you!,en


In [36]:
pq.write_table(pa.Table.from_pandas(twitter_content), "/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/TwitterData/df_all_tweets_with_langs.parquet")

In [7]:
pq.write_table(pa.Table.from_pandas(twitter_content), "/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/TwitterData/df_all_tweets_en.parquet")